In [1]:
import tensorflow as tf
tf.__version__

'2.4.1'

In [2]:
img_height = 28
img_width = 28
batch_size = 16

# Method 1
## custom dataset_from_directory

In [3]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    'data/',            # data folder
    labels='inferred',
    label_mode='int',   # categorical, binary
    # class_names = ['0', '1' ... ]
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height, img_width), #reshape if not this size
    shuffle=True,
    seed=1,  # if you want same stuff every time u run
    validation_split=0.1,
    subset='training'
)

ds_validation = tf.keras.preprocessing.image_dataset_from_directory(
    'data/',            # data folder
    labels='inferred',
    label_mode='int',   # categorical, binary
    # class_names = ['0', '1' ... ]
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=(img_height, img_width), #reshape if not this size
    shuffle=True,
    seed=1,  # if you want same stuff every time u run
    validation_split=0.1,
    subset='validation'
)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'data/'

## Augmentation the dataset

In [ ]:
def augment(x, y):
    image = tf.image.random_brightness(x, max_delta=0.05)
    return image, y

ds_train.map(augment())

## Compile and Train
model.compile( ... ) <br>
model.fit(train_ds, epochs=10, verbose=False) <br>

# Method 2
## ImageDataGenerato and flow_from_directory

In [ ]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range=(0.95, 0.95),
    horizontal_flip=False,
    vertical_flip=False,
    data_format='channels_last',
    validation_split=0.1,
    dtype=tf.float32
)

train_generator = data_gen.flow_from_directory('data/',
                                               target_size=(img_height, img_width),
                                               color_mode='grayscale',
                                               class_mode='sparse',
                                               shuffle=True,
                                               subset='training',
                                               seed=1)

val_generator = data_gen.flow_from_directory('data/',
                                             target_size=(img_height, img_width),
                                             color_mode='grayscale',
                                             class_mode='sparse',
                                             shuffle=True,
                                             subset='validation',
                                             seed=1)

## Compile and Train
model.compile( ... ) <br>
model.fit(train_generator, epochs=10, steps_per_epoch=num_images/batch_size, verbose=False)

# Method 3
## from csv file

In [ ]:
df = pd.read_csv('data/train.csv')

labels = df['label'].values
img_paths = df['img_paths'].values

ds_train = tf.data.Dataset.from_tensor_slices((img_paths, labels))
def read_image(img_path, label):
    image = tf.io.read_file(img_path)
    image = tf.image.decode_image(image, channels=1, dtype=tf.float32)

    return image, label

def augment(image, label):
    # data augment here
    return image, label

ds_train = ds_train.map(read_image).map(augment).batch(batch_size=batch_size)

## Compile and Train
model.compile( ... )
model.fit(ds_train, epochs=10, verbose=False)